### get standard libraries

In [381]:
import os.path
import errno
import json 
import datetime

### Get Google API Client libraries

In [382]:
# Google API Client Libraries
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials

### Set the configs needed for function

In [383]:
# Configuration
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]
CREDENTIALS_FILE = 'credentials.json'
TOKEN_FILE = 'token.json'
STATE_FILE = 'auth_state.json'
LOG_FILE = 'auth_log.json'

In [384]:
def authenticate_and_get_status():
    creds = None
    trigger_reason = None
    status = 'FAILURE'

    if os.path.exists(TOKEN_FILE):
        try:
            creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
            if not creds or not creds.valid:
                trigger_reason = 'TOKEN_INVALID'
                creds = None
            else:
                status = 'TOKEN_SUCCESS'
        except Exception as e:
            trigger_reason = 'TOKEN_LOAD_ERROR'
            creds = None
    else:
        trigger_reason = 'NO_TOKEN_FILE'
        creds = None

    if status != 'TOKEN_SUCCESS':
        try:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
            status = 'BROWSER_AUTH_SUCCESS'
            if trigger_reason is None:
                trigger_reason = 'INITIATED_BROWSER_FLOW'
            if creds:
                try:
                    with open(TOKEN_FILE, 'w') as token:
                        token.write(creds.to_json())
                except Exception as e:
                    pass
        except FileNotFoundError:
            trigger_reason = 'MISSING_CREDENTIALS_FILE'
            status = 'FAILURE'
            creds = None
        except Exception as e:
            trigger_reason = 'BROWSER_FLOW_ERROR'
            status = 'FAILURE'
            creds = None

    service = None
    if status in ['TOKEN_SUCCESS', 'BROWSER_AUTH_SUCCESS'] and creds and creds.valid:
        try:
            service = build('gmail', 'v1', credentials=creds)
        except HttpError as error:
            status = 'FAILURE'
            trigger_reason = 'BUILD_SERVICE_ERROR'
            service = None
        except Exception as e:
            status = 'FAILURE'
            trigger_reason = 'BUILD_SERVICE_ERROR'
            service = None
    elif status != 'FAILURE':
        status = 'FAILURE'
        if trigger_reason is None: trigger_reason = 'POST_AUTH_VALIDATION_FAILED'

    result = {'status': status, 'trigger_reason': trigger_reason}
    return service, result

In [385]:
#V0 code - not used now
def authenticate_with_token():
    creds = None
    print("starting core authenticaiton flow")
    
    if os.path.exists(TOKEN_FILE):
        print(f"Found {TOKEN_FILE}. Attempging to load credentials...")
        try:
            creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
            print("Loaded credentials from token.json")
        except Exception as e:
            print(f"Error loading {TOKEN_FILE}: {e}. Will have to proceed with browser authorizaiton now.")
            creds = None
    
    if not creds or not creds.valid:
        if creds:
            print("Token exists, but it is invalid. Have to Proceed with browser authorization again")
    
        try:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
            
            print("Authentication successful (user granted permission via browser).")
        
        except FileNotFoundError:
        
            print(f"Error: Credentials file {CREDENTIALS_FILE} not found.")
            return None
        
        except Exception as e:
            print(f"An error occurred during the authentication flow: {e}")
            return None
        
        if creds:
            try:
                with open(TOKEN_FILE, 'w') as token:
                    token.write(creds.to_json())
                print(f"Credentials saved to {TOKEN_FILE}")
            except Exception as e:
                print(f"Error saving token to {TOKEN_FILE}: {e}")

    
    if creds and creds.valid:
        try:
            service = build('gmail', 'v1', credentials=creds)
            print("Gmail API service object created successfully.")
            return service
        except HttpError as error:
            print(f"An HTTP error ocurred building the service: {error}")
            return None
        except Exception as e:
            print(f"An unexpected error occurred building the service: {e}")
            return None
    else:
        print("Authentication flow completed but no valid credentials obtained.")
        return None

In [386]:
def log_event(event_data):
    try:
        with open(LOG_FILE, 'a') as f:
            json.dump(event_data, f)
            f.write('\n')
    except IOError as e:
        print(f"Error writing log event to {LOG_FILE}: {e}")

In [387]:
if __name__ == '__main__':
    print("---Running authentication test (Stateful with JSON Logging)---")
    
    current_state = {
        'last_browser_attempt_id': None,
        'consecutive_token_successes': 0
    }
    
    
    
    try:
        with open(STATE_FILE, 'r') as f:
            current_state = json.load(f)
        print(f"Loaded previous state: {current_state}")
        
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"State file '{STATE_FILE} not found or invlaid, starting fresh state: {e}")
    
    
    
    timestamp_start = datetime.datetime.now(datetime.timezone.utc).isoformat()
    gmail_service, auth_result = authenticate_and_get_status()
    timestamp_end = datetime.datetime.now(datetime.timezone.utc).isoformat()
    
    
    log_entry = {
            'timestamp_start_utc': timestamp_start,
            'timestamp_end_utc': timestamp_end,
            'auth_status': auth_result['status'],
            'trigger_reason': auth_result['trigger_reason'],
            'browser_attempt_id': current_state.get('last_browser_attempt_id'),
            'token_success_count': None
    }    
    
    
    new_state = current_state.copy() # Start with old state

    if auth_result['status'] == 'TOKEN_SUCCESS':
        # Increment consecutive successes
        new_state['consecutive_token_successes'] = current_state.get('consecutive_token_successes', 0) + 1
        log_entry['token_success_count'] = new_state['consecutive_token_successes']
        print(f"Authentication successful using token (Consecutive success #{new_state['consecutive_token_successes']}).")

    elif auth_result['status'] == 'BROWSER_AUTH_SUCCESS':
        # Browser flow was used, potentially resetting the token success streak
        previous_successes = current_state.get('consecutive_token_successes', 0)
        new_browser_attempt_id = timestamp_start # Use start timestamp as unique ID for this browser session

        log_entry['browser_attempt_id'] = new_browser_attempt_id # Log the NEW ID
        log_entry['token_success_count'] = 0 # Reset count for logging this event

        new_state['last_browser_attempt_id'] = new_browser_attempt_id
        new_state['consecutive_token_successes'] = 0 # Reset streak counter

        print(f"Authentication successful via browser (New Browser Attempt ID: {new_browser_attempt_id}).")
        if previous_successes > 0:
            print(f"(Previous token streak ended after {previous_successes} successes due to: {auth_result['trigger_reason']})")
        else:
                print(f"(Browser flow triggered by: {auth_result['trigger_reason']})")


    else: # status == 'FAILURE'
        # Authentication failed
        previous_successes = current_state.get('consecutive_token_successes', 0)
        log_entry['token_success_count'] = 0 # Log 0 for failure event

        new_state['consecutive_token_successes'] = 0 # Reset streak counter on failure

        print(f"Authentication failed.")


---Running authentication test (Stateful with JSON Logging)---
Loaded previous state: {'last_browser_attempt_id': '2025-04-20T10:20:56.908106+00:00', 'Consecutive_token_successes': 0, 'consecutive_token_successes': 10}
Authentication successful using token (Consecutive success #11).


In [388]:
#         if previous_successes > 0:
#             print(f"(Process failed after {previous_successes} successful token uses. Reason: {auth_result['trigger_reason']})")
#         else:
#                 print(f"(Failure reason: {auth_result['trigger_reason']})")


#     # --- Log the Event ---
#     log_event(log_entry)

#     # --- Save Updated Persistent State ---
#     try:
#         with open(STATE_FILE, 'w') as f:
#             json.dump(new_state, f, indent=4) # Use indent for readability
#         print(f"Saved updated state: {new_state}")
#     except IOError as e:
#         print(f"Error writing state to {STATE_FILE}: {e}")
            
            
#     if gmail_service:
#             print("The 'gmail_service' object is ready.")
#     else:
#             print("Failed to get 'gmail_service' object this run.")

#     print(f"--- End of Authentication Test ---")

In [389]:
# OLD MAIN FUNCTION
# if __name__ == '__main__':
#     print("--Running Authentication Test (with Token Persistence--")
#     gmail_service = authenticate_with_token()
    
#     if gmail_service:
#         print("\n Core Authentication test Successful! The 'gmail_service' object is ready.")
#         print("Note: if token_json was used, browser authenticaiton was skipped.")
        
#     else:
#         print("\nCore Authentication test failed. Please check error messages above.")
        
#     print("--End of Authentication Test--")